In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyabf
import os
import pandas as pd


In [2]:
file = r"C:\Users\dongq\OneDrive\mouse patch-seq\Mecp2 project\3 weeks\2023_05_05_0001.abf"

In [3]:
f = pyabf.ABF(file)


In [4]:
f.sweepPointCount

20000

In [5]:
for index in f.sweepList:
    f.setSweep(index)
    print(f.sweepEpochs.levels, f.sweepEpochs.types)

[0.0, 0.0, -100.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, -80.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, -60.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, -40.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, -20.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 0.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 20.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 40.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 60.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 80.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 100.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 120.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 140.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 160.0, 0.0] ['Step', 'Step', 'Step', 'Step']
[0.0, 0.0, 180.0, 0.0] ['Step', 'Step', 'Step', 'Step']


In [6]:
f.sweepEpochs.p1s
f.sweepEpochs.p2s

[312, 1312, 11312, 20000]

In [7]:
f.setSweep(0)
first_epoc = f.sweepEpochs
f.setSweep(1)
second_epoc = f.sweepEpochs
dif = np.asarray(second_epoc.levels) - np.asarray(first_epoc.levels)



In [8]:
step = np.where(dif != 0)
start = np.asarray(first_epoc.p1s)[step][0]
end = np.asarray(first_epoc.p2s)[step][0]
(start, end)

(1312, 11312)

In [9]:
sampling_rate = f.sampleRate

In [10]:
from feature_extractor import SpikeFeatureExtractor, SpikeTrainFeatureExtractor
sfe = SpikeFeatureExtractor(filter=2)
spte = SpikeTrainFeatureExtractor(start=start/sampling_rate, end=end/sampling_rate)

In [11]:
import subthresh_features as sbth

In [12]:

rmp = []
tau = []
sag = []

t_set = []
i_set = []
v_set = []

temp_result_list = []

for index in f.sweepList:
    f.setSweep(index)
    t = f.sweepX
    v = f.sweepY
    i = f.sweepC

    if f.sweepUnitsY == f.sweepUnitsC:
        v = v/20
    current = np.asarray(f.sweepEpochs.levels)[step][0]

    rmp.append(np.median(v[np.where(i == 0)]))
    if current < 0:
        tau.append(sbth.time_constant(t=t, v=v, i=i, start=start/sampling_rate, end=(end-2)/sampling_rate))
        sag.append(sbth.sag(t=t, v=v, i=i, start=start/sampling_rate, end=end/sampling_rate))
        t_set.append(t)
        i_set.append(i)
        v_set.append(v)
    
    if current == 0:
        t_set.append(t)
        i_set.append(i)
        v_set.append(v)

    ft = sfe.process(t, v, i)
    # ft.to_csv(f'{index}.csv', index=False)
    sptft= spte.process(t=t, v=v, i=i, spikes_df=ft)
    print(sptft)
    sptft['current'] = current
    temp_result_list.append((ft, sptft))

rin = sbth.input_resistance(t_set=t_set, i_set=i_set, v_set=v_set, start=start/sampling_rate, end=end/sampling_rate)

  


{'avg_rate': 0}
{'avg_rate': 0}
{'avg_rate': 0}
{'avg_rate': 0}
{'avg_rate': 0}
{'avg_rate': 0}
{'avg_rate': 0}
{'adapt': 0.12672819428139087, 'latency': 0.0421, 'isi_cv': 0.521314477668212, 'mean_isi': 0.08516363636363637, 'median_isi': 0.08560000000000001, 'first_isi': 0.012199999999999989, 'avg_rate': 12.0}
{'adapt': 0.07080151363964594, 'latency': 0.019600000000000006, 'isi_cv': 0.40472829072871536, 'mean_isi': 0.06830714285714286, 'median_isi': 0.07325000000000001, 'first_isi': 0.01529999999999998, 'avg_rate': 15.0}
{'adapt': 0.06130885606455543, 'latency': 0.012999999999999984, 'isi_cv': 0.3997639638642004, 'mean_isi': 0.061381250000000005, 'median_isi': 0.06789999999999999, 'first_isi': 0.013200000000000017, 'avg_rate': 17.0}
{'adapt': 0.05245607182470689, 'latency': 0.009899999999999992, 'isi_cv': 0.37382095771317325, 'mean_isi': 0.0549764705882353, 'median_isi': 0.06040000000000001, 'first_isi': 0.012700000000000017, 'avg_rate': 18.0}
{'adapt': 0.045833296494450046, 'latency':

In [43]:
1508+1311


2819

## Todo:
volatge clamp mode